## Setup

In [ ]:
!pip install rouge/requirements.txt
!pip install rouge-score

ERROR: Invalid requirement: 'rouge/requirements.txt'
Hint: It looks like a path. File 'rouge/requirements.txt' does not exist.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=74b615797ee07ae337ae8986de9fedaab11a8c070623396e4da166def5729d24
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [ ]:
import random
import re
from rouge_score import rouge_scorer
import pandas as pd

In [ ]:
# remove extra chars

def cleantext(text):
  text = text.replace("\n"," ")
  text = text.split(" ")
  text = list(filter(lambda a: a != "", text))
  return " ".join(text)

## Baseline equations

In [ ]:
def first1k(text):
  cleaned = cleantext(text)
  summ = ""
  summ_len = 0
  for sent in re.split('\.|\?|\!', cleaned):
    summ_len+= len(sent.split(" "))
    if(summ_len>1000):
      break
    else:
      summ+= sent + "."
  
  return summ

def random_sentences(text):
  cleaned = cleantext(text)
  sentences = re.split('\.|\?|\!', cleaned)

  summ_length = 0
  bline = ""
  random.shuffle(sentences)
  for sent in sentences:
    cleansent = sent.split(" ")
    cleansent = list(filter(lambda a: a != "", cleansent))
    
    sentlen = len(cleansent)
    if(sentlen>5):
      summ_length+=sentlen
      if(summ_length>1000):
        break
      else:
        bline += (sent + ".")

  return bline

SUM_SENTENCES = 40

def chapter_starts(text):
  cleaned = cleantext(text)
  chaps_rough = re.split('Chapter|CHAPTER|chapter', cleaned)
  chaps = list(filter(lambda a: len(a)>2000, chaps_rough[1:]))
  if(len(chaps)<2):
    chaps=chaps_rough

  numchaps = len(chaps)
  numsentences = int(SUM_SENTENCES / numchaps) +1
  if(numsentences == 1): numsentences+=1

  bline =""
  for chap in chaps:
    cur = re.split('\.|\?|\!', chap)
    bline+= ".".join(cur[:numsentences])


  return bline

SUM_SENTENCES = 40

def chapter_ends(text):
  cleaned = cleantext(text)
  chaps_rough = re.split('Chapter|CHAPTER|chapter', cleaned)
  chaps = list(filter(lambda a: len(a)>2000, chaps_rough[1:]))
  if(len(chaps)<2):
    chaps=chaps_rough

  numchaps = len(chaps)
  numsentences = int(SUM_SENTENCES / numchaps)+1
  if(numsentences == 1): numsentences+=1

  bline =""
  for chap in chaps:
    cur = re.split('\.|\?|\!', chap)
    bline+= ".".join(cur[len(cur)-numsentences:])


  return bline

## Creating baseline on dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

os.chdir("/content/drive/MyDrive/practicum/summerizer/")
!ls

df = pd.read_csv('BookList_Long.csv', delimiter = "~").drop(columns = 'Unnamed: 0')

Mounted at /content/drive
 BookList_Long_theo.csv  'Copy of summerizer .ipynb'   new_booklist.csv


In [ ]:
df["first1k_text"] = df["full_text"].map(first1k)
df["random_text"] = df["full_text"].map(random_sentences)
df["chapter_start_text"] = df["full_text"].map(chapter_starts)
df["chapter_end_text"] = df["full_text"].map(chapter_ends)


In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)

In [ ]:
scorer.score("Harry goes to the sea and kills off a vendor","Harry goes to the market and pays off a vendor.")

{'rouge1': Score(precision=0.8, recall=0.8, fmeasure=0.8000000000000002),
 'rouge2': Score(precision=0.5555555555555556, recall=0.5555555555555556, fmeasure=0.5555555555555556),
 'rougeL': Score(precision=0.8, recall=0.8, fmeasure=0.8000000000000002)}

In [ ]:
scorer.score("Harry settles his debts at the bazaar with one of the vendors", "Harry goes to the market and pays off a vendor.")



{'rouge1': Score(precision=0.3, recall=0.25, fmeasure=0.2727272727272727),
 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0),
 'rougeL': Score(precision=0.3, recall=0.25, fmeasure=0.2727272727272727)}

In [ ]:
df["first1k_rouge"]= df.apply(lambda x: scorer.score(x['first1k_text'], x['summary']), axis=1)
df["random_rouge"]= df.apply(lambda x: scorer.score(x['random_text'], x['summary']), axis=1)
df["chapter_start_rouge"]= df.apply(lambda x: scorer.score(x['chapter_start_text'], x['summary']), axis=1)
df["chapter_end_rouge"]= df.apply(lambda x: scorer.score(x['chapter_end_text'], x['summary']), axis=1)


In [ ]:
df["first1k_rouge"]

0      {'rouge1': (0.3997194950911641, 0.287588294651...
1      {'rouge1': (0.34526558891454967, 0.32429501084...
2      {'rouge1': (0.39544235924932974, 0.30793319415...
3      {'rouge1': (0.4316644113667118, 0.335084033613...
4      {'rouge1': (0.48427672955974843, 0.24470338983...
                             ...                        
98     {'rouge1': (0.45075757575757575, 0.12306101344...
99     {'rouge1': (0.5158371040723982, 0.119496855345...
100    {'rouge1': (0.45286506469500926, 0.49949031600...
101    {'rouge1': (0.4347202295552367, 0.310769230769...
102    {'rouge1': (0.5416666666666666, 0.079268292682...
Name: first1k_rouge, Length: 103, dtype: object

## Evaluating baselines

In [ ]:
def unpack_rouge(column_name):
  df[column_name+"1"] = df[column_name].map(lambda x: x["rouge1"].fmeasure) 
  df[column_name+"2"] = df[column_name].map(lambda x: x["rouge2"].fmeasure) 
  df[column_name+"L"] = df[column_name].map(lambda x: x["rougeL"].fmeasure) 

In [ ]:
unpack_rouge("first1k_rouge")
unpack_rouge("random_rouge")
unpack_rouge("chapter_start_rouge")
unpack_rouge("chapter_end_rouge")

In [ ]:
df.describe()

,first1k_rouge1,first1k_rouge2,first1k_rougeL,random_rouge1,random_rouge2,random_rougeL,chapter_start_rouge1,chapter_start_rouge2,chapter_start_rougeL,chapter_end_rouge1,chapter_end_rouge2,chapter_end_rougeL
count,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000
mean,0.334926,0.053696,0.133023,0.351426,0.052238,0.130084,0.316701,0.055235,0.131633,0.334143,0.056781,0.136601
std,0.078399,0.021172,0.022940,0.086045,0.023539,0.021500,0.083853,0.022846,0.024497,0.080700,0.041578,0.039740
min,0.116775,0.011142,0.067797,0.095735,0.012467,0.057441,0.127702,0.014423,0.076420,0.130283,0.006944,0.074293
25%,0.298405,0.040102,0.123259,0.312221,0.038341,0.123450,0.261290,0.039110,0.121184,0.291427,0.039326,0.120711
50%,0.352211,0.050756,0.134977,0.378811,0.050975,0.134857,0.326250,0.049955,0.133750,0.347211,0.052928,0.136341
75%,0.389663,0.064987,0.144433,0.414944,0.062717,0.142420,0.378717,0.068463,0.144071,0.382908,0.063890,0.149225
max,0.475036,0.142500,0.224719,0.508524,0.183477,0.185095,0.497772,0.128262,0.235294,0.649190,0.421756,0.455672


In [ ]:
df.drop(["first1k_rouge", "random_rouge","chapter_start_rouge","chapter_end_rouge"], axis=1, inplace = True)

## Saving

In [ ]:
df.to_csv('new_booklist.csv', sep ='~') 

!cp new_booklist.csv "/content/drive/MyDrive/practicum/summerizer"